# Titanic

# Załącznik

In [33]:
import numpy as np 
import pandas as pd 
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE 
import sklearn
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import sys
import numpy as np
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [34]:
df = pd.read_csv('titanic.csv')
df['Age'] = df['Age'].apply(lambda x: int(x >= 18))
df['Sex'] = df['Sex'].apply(lambda x: int(x == 'male'))
df.drop(['Name'], axis=1, inplace=True)
Y = df['Survived']
X = df.drop(['Survived'], axis=1)
train_X, test_X, train_Y, test_Y = train_test_split(X, Y,  
                      test_size = 0.3, random_state = 123) 

In [35]:

df

,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,1,1,1,0,7.2500
1,1,1,0,1,1,0,71.2833
2,1,3,0,1,0,0,7.9250
3,1,1,0,1,1,0,53.1000
4,0,3,1,1,0,0,8.0500
...,...,...,...,...,...,...,...
882,0,2,1,1,0,0,13.0000
883,1,1,0,1,0,0,30.0000
884,0,3,0,0,1,2,23.4500
885,1,1,1,1,0,0,30.0000


In [136]:
def train_model(model, prot_att, privileged_group, unprivileged_group):
    global train_X, test_X, train_Y, test_Y
    model.fit(train_X, train_Y)
    pred = model.predict(test_X)
    test_X_prd = test_X.copy()
    test_X_prd['Survived'] = pred
    test_X_gt = test_X.copy()
    test_X_gt['Survived'] = test_Y
    df_aifGT = BinaryLabelDataset(df=test_X_gt, label_names=['Survived'], protected_attribute_names=[prot_att])
    df_aifPRD = BinaryLabelDataset(df=test_X_prd, label_names=['Survived'], protected_attribute_names=[prot_att])
    metric_orig_test = ClassificationMetric(df_aifGT, df_aifPRD, unprivileged_group, privileged_group)
    return metric_orig_test

def print_metric(metric):
    selecRT = metric.selection_rate(privileged=True)/metric.selection_rate(privileged=False)
    print ('statistical_parity_ratio', min(selecRT, 1/selecRT))
#     print(metric.statistical_parity_difference())
#     print(metric.true_positive_rate())

    print('equal opportunity ratio', metric.false_negative_rate_ratio())
    print(metric.equal_opportunity_difference())
#     print(metric.true_positive_rate())
    
#     print(metric.false_positive_rate_difference())
    print('predictive equality ratio', metric.false_positive_rate_ratio())
#     print(metric.false_positive_rate())



In [145]:
def statiPari(metric):
    selecRT = metric.selection_rate(privileged=True)/metric.selection_rate(privileged=False)
    return min(selecRT, 1/selecRT)

def PER(metric):
    mtr = metric.false_positive_rate_ratio()
    return min(mtr, 1/mtr)

In [146]:
columns=["Sex", "Age", "Pclass"]
dfSPD = pd.DataFrame(columns=columns)
dfSPR = pd.DataFrame(columns=columns)
dfEQD = pd.DataFrame(columns=columns)
dfEQR = pd.DataFrame(columns=columns)
dfPER = pd.DataFrame(columns=columns)
def gen_plots(model, desc):
    global dfSPD, dfSPR, dfEQD, dfEQR, dfPER
    metric1 = train_model(model, "Sex", [{'Sex': 1}], [{'Sex': 0}])
    
    print_metric(metric1)
    metric2 = train_model(model, "Age", [{'Age': 1}], [{'Age': 0}])
    print_metric(metric2)
    metric3 = train_model(model, "Pclass", [{'Pclass': 1}], [{'Pclass': 3, 'Pclass': 2}])
    print_metric(metric3)
    dfSPD = pd.concat([dfSPD, pd.DataFrame([[metric1.statistical_parity_difference(), 
                  metric2.statistical_parity_difference(), 
                  metric3.statistical_parity_difference()]], index = [desc], columns=columns)])
    dfSPR = pd.concat([dfSPR, pd.DataFrame([[statiPari(metric1), 
                  statiPari(metric2), 
                  statiPari(metric3)]], index = [desc], columns=columns)])
    dfEQD = pd.concat([dfEQD, pd.DataFrame([[metric1.equal_opportunity_difference(), 
                  metric2.equal_opportunity_difference(), 
                  metric3.equal_opportunity_difference()]], index = [desc], columns=columns)])
    dfEQR = pd.concat([dfEQR, pd.DataFrame([[metric1.false_negative_rate_ratio(), 
                  metric2.false_negative_rate_ratio(), 
                  metric3.false_negative_rate_ratio()]], index = [desc], columns=columns)])
    dfPER = pd.concat([dfPER, pd.DataFrame([[PER(metric1), 
                  PER(metric2), 
                  PER(metric3)]], index = [desc], columns=columns)])

In [147]:
gen_plots(xgb.XGBClassifier(), 'XGB')
gen_plots(RandomForestClassifier(), 'RF')
gen_plots(xgb.XGBClassifier(max_depth=2), 'XGB (max_depth=2)')

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:26:26] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
statistical_parity_ratio 0.17863647161295432
equal opportunity ratio 0.23578947368421052
0.5185714285714285
predictive equality ratio 5.087719298245614
[16:26:26] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
statistical_parity_ratio 0.5677777777777778
equal opportunity ratio 0.91
0.027692307692307683
predictive equality ratio 2.7306501547987616
[16:26:26] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_met

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


In [148]:
dfSPD

,Sex,Age,Pclass
XGB,0.637867,0.246984,-0.040265
RF,0.653364,0.299048,0.010066
XGB (max_depth=2),0.727586,0.165079,-0.055508


In [149]:
dfSPR

,Sex,Age,Pclass
XGB,0.178636,0.567778,0.934426
RF,0.181118,0.516923,0.983404
XGB (max_depth=2),0.100091,0.653333,0.906943


In [150]:
dfEQD

,Sex,Age,Pclass
XGB,0.518571,0.027692,0.090756
RF,0.424762,0.134872,0.147059
XGB (max_depth=2),0.580952,0.014872,0.062185


In [151]:
dfEQR

,Sex,Age,Pclass
XGB,0.235789,0.910000,0.485714
RF,0.256667,0.542609,0.000000
XGB (max_depth=2),0.186667,0.949565,0.647619


In [152]:
dfPER

,Sex,Age,Pclass
XGB,0.196552,0.366213,0.379121
RF,0.154859,0.393197,0.353846
XGB (max_depth=2),0.071473,0.867347,0.294872
